In [2]:
import pandas as pd
from datetime import date
from src import get_soup_obj
import re

In [22]:
df = pd.read_csv('data/rtlbmp_zone_1999-2024.csv')
# df.set_index(['Time Stamp', 'Name'], inplace=True)

In [28]:
df

,Time Stamp,Name,PTID,LBMP ($/MWHr),Marginal Cost Losses ($/MWHr),Marginal Cost Congestion ($/MWHr)
0,1999-11-18 00:00:00,CAPITL,61757,8.65,0.54,0.0
1,1999-11-18 00:00:00,CENTRL,61754,7.98,-0.13,0.0
2,1999-11-18 00:00:00,DUNWOD,61760,8.46,0.35,0.0
3,1999-11-18 00:00:00,GENESE,61753,8.02,-0.09,0.0
4,1999-11-18 00:00:00,H Q,61844,8.05,-0.06,0.0
...,...,...,...,...,...,...
41721910,2024-05-19 18:00:00,NORTH,61755,22.84,0.14,0.0
41721911,2024-05-19 18:00:00,NPX,61845,23.57,0.86,0.0
41721912,2024-05-19 18:00:00,O H,61846,21.96,-0.75,0.0
41721913,2024-05-19 18:00:00,PJM,61847,23.18,0.48,0.0


In [23]:
latest_date_in_database_split = df['Time Stamp'].iloc[-1].split(' ')[0].split('-')
latest_date_in_database = latest_date_in_database_split[1] + '-' + latest_date_in_database_split[2] + '-' + latest_date_in_database_split[0]

In [24]:
latest_date_in_database

'05-19-2024'

In [25]:
url = 'http://mis.nyiso.com/public/P-24Alist.htm'
soup = get_soup_obj(url)

recent_csvs = soup.find_all('a', href=re.compile(".csv$"))

csv_index_to_download_till = None

for i, link in enumerate(recent_csvs):
     if latest_date_in_database in link.text:
         csv_index_to_download_till = i

if csv_index_to_download_till == None:
     print("Error: Last date not found in the list of available csvs")

if csv_index_to_download_till == 0:
     print("No new data to download")

Error: Last date not found in the list of available csvs


In [26]:
soup.find_all('a', href=re.compile(".csv$"))

[<a href="realtime/realtime_zone_lbmp.csv" target="_parent">Most recent interval</a>,
 <a href="csv/realtime/20240529realtime_zone.csv" target="_parent">05-29-2024</a>,
 <a href="csv/realtime/20240528realtime_zone.csv" target="_parent">05-28-2024</a>,
 <a href="csv/realtime/20240527realtime_zone.csv" target="_parent">05-27-2024</a>,
 <a href="csv/realtime/20240526realtime_zone.csv" target="_parent">05-26-2024</a>,
 <a href="csv/realtime/20240525realtime_zone.csv" target="_parent">05-25-2024</a>,
 <a href="csv/realtime/20240524realtime_zone.csv" target="_parent">05-24-2024</a>,
 <a href="csv/realtime/20240523realtime_zone.csv" target="_parent">05-23-2024</a>,
 <a href="csv/realtime/20240522realtime_zone.csv" target="_parent">05-22-2024</a>,
 <a href="csv/realtime/20240521realtime_zone.csv" target="_parent">05-21-2024</a>,
 <a href="csv/realtime/20240520realtime_zone.csv" target="_parent">05-20-2024</a>]

In [18]:
csv_urls_to_download = []

for link in recent_csvs[2:csv_index_to_download_till]:
    csv_urls_to_download.append('http://mis.nyiso.com/public' + "/" + link['href'])

csv_urls_to_download

['http://mis.nyiso.com/public/csv/damlbmp/20240529damlbmp_gen.csv',
 'http://mis.nyiso.com/public/csv/damlbmp/20240528damlbmp_gen.csv',
 'http://mis.nyiso.com/public/csv/damlbmp/20240527damlbmp_gen.csv',
 'http://mis.nyiso.com/public/csv/damlbmp/20240526damlbmp_gen.csv',
 'http://mis.nyiso.com/public/csv/damlbmp/20240525damlbmp_gen.csv',
 'http://mis.nyiso.com/public/csv/damlbmp/20240524damlbmp_gen.csv',
 'http://mis.nyiso.com/public/csv/damlbmp/20240523damlbmp_gen.csv',
 'http://mis.nyiso.com/public/csv/damlbmp/20240522damlbmp_gen.csv',
 'http://mis.nyiso.com/public/csv/damlbmp/20240521damlbmp_gen.csv']

In [186]:
new_data = []

for url in csv_urls_to_download:
    new_data.append(pd.read_csv(url))

df_new_data = pd.concat(new_data)

df_new_data["Time Stamp"] = pd.to_datetime(df_new_data["Time Stamp"], format="%m/%d/%Y %H:%M")
df_new_data.set_index(['Time Stamp', 'Name'], inplace=True)
df_new_data = df_new_data.sort_values(by=["Time Stamp", "Name"], ascending=[True, True])

In [187]:
df_new_data

PTID  LBMP ($/MWHr)  \
Time Stamp          Name                           
2024-05-21 00:00:00 CAPITL  61757          22.77   
                    CENTRL  61754          22.03   
                    DUNWOD  61760          23.29   
                    GENESE  61753          21.79   
                    H Q     61844          21.42   
...                           ...            ...   
2024-05-29 23:00:00 NORTH   61755          22.36   
                    NPX     61845          25.20   
                    O H     61846          23.88   
                    PJM     61847          24.91   
                    WEST    61752          24.42   

                            Marginal Cost Losses ($/MWHr)  \
Time Stamp          Name                                    
2024-05-21 00:00:00 CAPITL                           0.98   
                    CENTRL                           0.24   
                    DUNWOD                           1.50   
                    GENESE                           0.00   
                    H Q                             -0.28   
...                                                   ...   
2024-05-29 23:00:00 NORTH                           -2.16   
                    NPX                              0.69   
                    O H                             -0.64   
                    PJM                              0.39   
                    WEST                            -0.10   

                            Marginal Cost Congestion ($/MWHr)  
Time Stamp          Name                                       
2024-05-21 00:00:00 CAPITL                               0.00  
                    CENTRL                               0.00  
                    DUNWOD                               0.00  
                    GENESE                               0.00  
                    H Q                                  0.09  
...                                                       ...  
2024-05-29 23:00:00 NORTH                                0.00  
                    NPX                                  0.00  
                    O H                                  0.00  
                    PJM                                  0.00  
                    WEST                                 0.00  

[3240 rows x 4 columns]

In [188]:
df

PTID  LBMP ($/MWHr)  \
Time Stamp          Name                           
1999-11-18 00:00:00 CAPITL  61757          28.21   
                    CENTRL  61754          26.22   
                    DUNWOD  61760          29.13   
                    GENESE  61753          26.07   
                    H Q     61844          27.79   
...                           ...            ...   
2024-05-20 23:00:00 NORTH   61755          23.33   
                    NPX     61845          24.27   
                    O H     61846          22.93   
                    PJM     61847          23.87   
                    WEST    61752          23.40   

                            Marginal Cost Losses ($/MWHr)  \
Time Stamp          Name                                    
1999-11-18 00:00:00 CAPITL                           1.23   
                    CENTRL                          -0.76   
                    DUNWOD                           2.15   
                    GENESE                          -0.91   
                    H Q                              0.81   
...                                                   ...   
2024-05-20 23:00:00 NORTH                           -0.07   
                    NPX                              0.87   
                    O H                             -0.47   
                    PJM                              0.47   
                    WEST                             0.00   

                            Marginal Cost Congestion ($/MWHr)  
Time Stamp          Name                                       
1999-11-18 00:00:00 CAPITL                                0.0  
                    CENTRL                                0.0  
                    DUNWOD                                0.0  
                    GENESE                                0.0  
                    H Q                                   0.0  
...                                                       ...  
2024-05-20 23:00:00 NORTH                                 0.0  
                    NPX                                   0.0  
                    O H                                   0.0  
                    PJM                                   0.0  
                    WEST                                  0.0  

[3222345 rows x 4 columns]

In [190]:
pd.concat([df, df_new_data]).to_csv("data/damlbmp_zone_1999-2024.csv")